In [5]:
import pandas as pd

df = pd.read_csv("a1_data.csv")

In [6]:
import numpy as np

X = df.values

In [7]:
import numpy as np
from sklearn.linear_model import LinearRegression, LassoLarsIC

def sortnregress(X):
    LR = LinearRegression()
    LL = LassoLarsIC(criterion='bic')

    d = X.shape[1]
    W = np.zeros((d, d))
    increasing = np.argsort(np.var(X, axis=0))
    print(increasing)
    for k in range(1, d):
        covariates = increasing[:k]
        target = increasing[k]
        LR.fit(X[:, covariates], X[:, target].ravel())
        weight = np.abs(LR.coef_)
        LL.fit(X[:, covariates] * weight, X[:, target].ravel())
        W[covariates, target] = LL.coef_ * weight
    return W


In [8]:
W = sortnregress(X)

[ 1  0  7  2  8 10  6  4  5  3  9]


In [9]:
# visualize the matrix as a directed graph
import networkx as nx
import matplotlib.pyplot as plt

G = nx.DiGraph(W)
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True)
plt.show()

In [10]:
import numpy as np
from sklearn.linear_model import Lasso

def sortnregress(X):
    lasso = Lasso()

    d = X.shape[1]
    W = np.zeros((d, d))
    increasing = np.argsort(np.var(X, axis=0))
    print(increasing)
    for k in range(1, d):
        covariates = increasing[:k]
        target = increasing[k]
        lasso.fit(X[:, covariates], X[:, target].ravel())
        W[covariates, target] = lasso.coef_
    return W